In [1]:
import pandas as pd
import numpy as np

In [2]:
INTERN = 'Data Scientist Intern'
JR = 'Junior Data Scientist'
SR = 'Senior Data Scientist'
DS = 'Data Scientist'
LDS = 'Lead Data Scientist'
rm = np.nan

In [3]:
def clean_position_salarydf(country):
    # rename 'position' to 'Position' column (for easier merging dfs later on)
    FILE = 'salaries_' + country + '.csv'
    df = pd.read_csv('../../../data/clean_data/' + FILE, index_col = 0)
    df = df.rename(columns={"position": "Position"})
    # duplicate
    df['Position_clean'] = df.loc[:, 'Position']
    # step 1: intern
    if country == 'us':
        df.loc[df['Position'] == 'Senior Data Scientist - Monthly Intern', 'Position_clean'] = SR
        df.loc[df['Position'] == 'Data Scientist II - Hourly Intern', 'Position_clean'] = SR
        df.loc[df['Position'] == 'Data Scientist III - Hourly Intern', 'Position_clean'] = SR
    s = ['intern', 'co-op', 'coop', 'placem']
    cond = '|'.join(s)
    bool_intern = (df['Position_clean'].str.contains(cond, case = False))& \
                (df['Position_clean'].str.contains("data scien", case = False)) 
    df.loc[bool_intern, 'Position_clean'] = INTERN
    # step 2: keyword 'data science'
        # lead ds
    bool_lds = (df['Position_clean'].str.contains('Manage|Direct|Lead|Head|VP|Principal|Chief', case = False)) & \
           (df['Position_clean'].str.contains('data science', case = False))
    df.loc[bool_lds, 'Position_clean'] = LDS
        # sr ds
    bool_sr = (df['Position_clean'].str.contains('Senior|Sr|intermediate|st II|graduate|staff|st 2', case = False)) &\
            (df['Position_clean'].str.contains('data science', case = False))
    df.loc[bool_sr, 'Position_clean'] = SR
        # jr ds
    bool_jr = (df['Position_clean'].str.contains('junior|jr|entry|st 1', case = False)) &\
            (df['Position_clean'].str.contains('data science', case = False))
    df.loc[bool_jr, 'Position_clean'] = JR
        # ds
    bool_ds = (df['Position_clean'].str.contains('Specialist|Analyst|Consultant|Scientist|Engineer|associate', case = False)) &\
            (df['Position_clean'].str.contains('data science', case = False))
    df.loc[bool_ds, 'Position_clean'] = DS
    # step 3-4: skip if remaining roles all contain keyword 'Data Scientist' 
    # step 5: special cases
    if country == 'us':
        df.loc[df['Position'] == 'Data and Applied Scientist', 'Position_clean'] = DS
    if country == 'uk':
        df.loc[df['Position'] == 'Daata Scientist', 'Position_clean'] = DS
    # step 6
    #bool_notDS = df['Position_clean'].str.contains('Data Scientist', case = False) == False
    # step 7
    # LDS
    bool_lds = df['Position_clean'].str.contains('Manage|Direct|Lead|Head|VP|Principal|Chief', case = False) 
    df.loc[ bool_lds, 'Position_clean']= LDS
    # SR 
    bool_sr = df['Position_clean'].str.contains('Senior|Sr|intermediate|st II|graduate|staff|st 2', case = False)
    df.loc[bool_sr, 'Position_clean'] = SR
    # JR
    bool_jr = df['Position_clean'].str.contains('junior|jr|entry|st 1', case = False) 
    df.loc[ bool_jr, 'Position_clean']= JR
    # DS
    bool_remain = (df['Position_clean'].str.contains('senior', case = False, na = False) == False) &\
                (df['Position_clean'].str.contains('junior', case = False, na = False) == False) &\
                (df['Position_clean'].str.contains('lead', case = False, na = False) == False) &\
                (df['Position_clean'].str.contains('intern', case = False, na = False) == False) &\
                df['Position_clean'].notna()
    df.loc[bool_remain, 'Position_clean'] = DS
    
    # drop 'Position' column
    df = df.drop(['Position'], axis=1)
    # write to .csv
    NEWFILE = 'salaries_' + country + '2' + '.csv'
    df.to_csv('../../../data/clean_data/' + NEWFILE)
    # return 
    return df
    

In [4]:
df1 = clean_position_salarydf('us')

In [5]:
df2 = clean_position_salarydf('ca')

In [6]:
df3 = clean_position_salarydf('uk')

In [8]:
df1.head()

,company,salary,lower_bound_salary,upper_bound_salary,base_pay,bonus_pay,cash_bonus,stock_bonus,Position_clean
0,Ascendum Solutions,120294.68,120520.0,120520.0,120294.68,NaN,NaN,NaN,Data Scientist
1,Facebook,198153.22,131000.0,286890.0,198153.22,19650.00,19650.00,33267.45,Data Scientist
2,Expedia Group,158744.49,145410.0,187330.0,158744.49,43235.24,43235.24,24943.71,Data Scientist
3,"Quora, Inc.",155663.37,144100.0,183400.0,155663.37,NaN,NaN,NaN,Data Scientist
4,Oracle,183544.10,34060.0,251520.0,183544.10,20290.59,20290.59,15097.75,Data Scientist


In [9]:
df2.head()

,company,salary,lower_bound_salary,upper_bound_salary,base_pay,bonus_pay,cash_bonus,stock_bonus,Position_clean
0,Capital One,76883,73000,125000,76883.0,5885.0,5885.0,3272.0,Data Scientist
1,RBC,90000,55000,135000,90000.0,12589.0,11983.0,3147.0,Data Scientist
2,Scotiabank,96176,74000,103000,96176.0,10051.0,10051.0,3327.0,Data Scientist
3,TD,79473,50000,129000,79473.0,6062.0,6062.0,2393.0,Data Scientist
4,Banque Nationale du Canada/National Bank of Ca...,74945,71000,90000,74945.0,5045.0,5045.0,1536.0,Data Scientist


In [10]:
df3.head()

,company,salary,lower_bound_salary,upper_bound_salary,base_pay,bonus_pay,cash_bonus,stock_bonus,Position_clean
0,First Derivatives,NaN,33060.0,40020.0,35518.62,NaN,NaN,NaN,Data Scientist
1,Deliveroo,NaN,66120.0,226200.0,110952.84,NaN,NaN,NaN,Data Scientist
2,Barclays,NaN,69600.0,133980.0,92635.86,5569.74,5569.74,NaN,Data Scientist
3,Lloyds Banking Group,NaN,34800.0,182700.0,103016.70,13907.82,13907.82,5258.28,Data Scientist
4,THG,NaN,60900.0,88740.0,72545.82,6270.96,6270.96,NaN,Data Scientist


In [11]:
df1.Position_clean.unique()

array(['Data Scientist', 'Data Scientist Intern', 'Senior Data Scientist',
       'Lead Data Scientist', 'Junior Data Scientist'], dtype=object)